In [1]:
import numpy as np
import scipy.stats as stat
import matplotlib.pyplot as plot
import statsmodels.api as sm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# parameters declaration and data generation
n1, n2, n are w/e in range [100,1000]
q1 is w/e in range [0,1]
mu1,mu2 are w/e 3 dim vectors
sigma is w/e 3x3 matrix

In [8]:
def gen_sigma():
    all_eigens_pos = False
    while not all_eigens_pos:
        tmp = np.reshape( stat.uniform.rvs(loc=3, scale = 9, size=9), (3,3)) 
        sigma = (tmp + tmp.T) / 2
        all_eigens_pos = np.all(np.linalg.eigvals(sigma) > 0)
    return sigma

In [9]:
n1 = 300
n2 = 200
n = 100

q1 = n1 / (n1 + n2) 
q2 = 1 - q1

mu1 = stat.uniform.rvs(loc=-10, scale = 20, size=3) # uniform on [loc, loc+scale]
mu2 = mu1 + 0.1 * stat.uniform.rvs(loc=-5, scale = 15, size=3)
sigma = gen_sigma()

print(mu1)
print(mu2)
print(sigma)

print(q1, q2)

[ 3.75792221  4.28085952 -5.808463  ]
[ 4.61633684  5.04216241 -6.06192292]
[[10.99816306  8.08920341  7.92585309]
 [ 8.08920341 10.76510816  7.67847262]
 [ 7.92585309  7.67847262  7.25857834]]
0.6 0.4


In [10]:
sample1 = np.random.multivariate_normal(mu1, sigma, n1)
sample2 = np.random.multivariate_normal(mu2, sigma, n2)

test_sample1 = np.random.multivariate_normal(mu1, sigma, int(n * q1))
test_sample2 = np.random.multivariate_normal(mu2, sigma, int(n * q2))
test_sample = np.concatenate((test_sample1, test_sample2), axis=0)
# np.random.shuffle(test_sample)

# 1. parameters estimations

In [11]:
# n - sample size, p - dimension of each sample's entry
def estimate_mean(data):
    n = np.shape(data)[0]
    return np.sum(data, axis=0) / n

# ( ((n1 - 1) * np.cov(sample1.T) + (n2 - 2) * np.cov(sample2.T)) / (n1 + n2 - 2) )
def estimate_sigma(data1, data2, p=3):
    sigma = np.resize(np.zeros(p**2), new_shape=(p,p))
    for data in [data1, data2]:
        mu_k = estimate_mean(data)
        for l in range(p):
            for j in range(p):
                sigma[l][j] += np.sum((data[:,l] - mu_k[l]) * (data[:,j] - mu_k[j]))
    return sigma / (np.shape(data1)[0] + np.shape(data2)[0] - 2)

def estimate_alpha(sigma, mu1, mu2):
    return np.linalg.solve(sigma, mu1-mu2)

def calc_z(data, alpha):
    z = data @ alpha
    return np.sum(z) / np.shape(z)[0]

def calc_s_z(alpha, sigma):
    return alpha.T @ sigma @ alpha

def classify(data, z1, z2, q1, q2, alpha):
    class_1 = []
    class_2 = []
    class_num = []
    for entry in data:
        if alpha.T @ entry >= (z1 + z2) / 2 + np.log(q2 / q1):
            class_1.append(entry)
            class_num.append(1)
        else:
            class_2.append(entry)
            class_num.append(2)
    return class_1, class_2, class_num


est_mu1 = estimate_mean(sample1)
est_mu2 = estimate_mean(sample2)
est_sigma = estimate_sigma(sample1, sample2)
est_alpha = estimate_alpha(est_sigma, est_mu1, est_mu2)

z1 = calc_z(sample1, est_alpha)
z2 = calc_z(sample2, est_alpha)
cl_1, cl_2, pred_class_num = classify(test_sample, z1, z2, q1, q2, est_alpha)

real_class_num = np.concatenate((np.ones(int(n * q1)), 2 * np.ones(int(n * q2))))

print(real_class_num == pred_class_num) 


[False False  True  True False  True False False  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True  True False False  True  True  True False  True False  True  True
 False  True  True  True  True  True  True  True  True False  True  True
  True False  True  True  True  True  True  True False  True  True False
  True  True False False  True False False False False  True  True  True
  True False  True  True False  True False False  True False  True  True
  True False False False False  True False False False  True  True False
  True  True False False]


# 2 estimate classification error probability

In [12]:
def mahalanobis_distance(z1, z2, s_z):
    return (z1 - z2)**2 / s_z

def unbiased_mahalanobis_distance(z1, z2, s_z, n1, n2, p=3):
    D = mahalanobis_distance(z1, z2, s_z)
    return (n1 + n2 - p - 3) / (n1 + n2 - 2) * D**2 - p * (1 / n1 + 1 / n2)

s_z = calc_s_z(est_alpha, est_sigma)
D = mahalanobis_distance(z1, z2, s_z)
D_H = unbiased_mahalanobis_distance(z1, z2, s_z, n1, n2)
print(D)
print(D_H)

1.5799039009430598
2.451047369659153


In [13]:
K = np.log(q2 / q1)
prob_2_1 = stat.norm.cdf((K - 0.5 * D_H**2 ) / D_H)
prob_1_2 = stat.norm.cdf((-K - 0.5 * D_H**2 ) / D_H)
print(prob_2_1)
print(prob_1_2)

0.08212045711948579
0.14454990843327675


# 3 classification of test sample

In [14]:

def print_error_table(predict, actual):
    table = [[0,0],[0,0]]
    n = np.shape(predict)[0]
    for i in range(n):
        if predict[i] == 1. and actual[i] == 1.:
            table[0][0] += 1/n
        elif predict[i] == 2. and actual[i] == 1.:
            table[0][1] += 1/n
        elif predict[i] == 1. and actual[i] == 2.:
            table[1][0] += 1/n
        else:
            table[1][1] += 1/n
    for true_vals in table:
        print(true_vals)
    return table

#            | predicted 1 | predicted 2 
# true val 1 |    [0,0]    |   [0,1]
# true val 1 |    [1,0]    |   [1,1]
table = print_error_table(pred_class_num, real_class_num)

[0.45000000000000023, 0.15]
[0.21000000000000005, 0.19000000000000003]


# 4 german dataset task

In [15]:
df = pd.read_csv("german.data", sep = ' ', skipinitialspace=True, header=None, dtype=np.float64)
df = df.drop([25], axis=1)
data = df.to_numpy()
# print(data)
# y = data[:,-1]
# data = np.delete(data, obj=-1, axis=1)

train, test = train_test_split(data, test_size=0.2, random_state=4)

def select_train_by_class(train, clazz):
    train_i = train[train[:,-1] == clazz]
    train_i = np.delete(train_i, obj=-1, axis=1)
    return train_i

def process_test(test):
    test = test[test[:, -1].argsort()] # sort by class value so it's easy to see correctly guessed classes
    n = np.shape(test)[0]
    q1 = np.shape(test[test[:,-1] == 1])[0] / n
    q2 = np.shape(test[test[:,-1] == 2])[0] / n
    test =  np.delete(test, obj=-1, axis=1)
    return test, q1, q2, n
    
def normalize(data):
    from copy import deepcopy
    data_t = deepcopy(data.T)
    for i in range(np.shape(data_t)[0]):
        x = data_t[i,:]
        max = np.max(x)
        min = np.min(x)
        data_t[i,:] = (x - min) / (max - min)
    return data_t.T

train_1 = select_train_by_class(train, 1)
# train_1_normilized = normalize(train_1)
# train_1 = train_1_normilized
n1 = np.shape(train_1)[0]

train_2 = select_train_by_class(train, 2)
n2 = np.shape(train_2)[0]
# train_2_normilized = normalize(train_2)
# train_2 = train_2_normilized 

q1 = n1 / (n1 + n2)
q2 = 1 - q1

test, tru_q1, tru_q2, n = process_test(test)
# test_norm = normalize(test)
# test = test_norm

p = np.shape(train_1)[1]

In [16]:
print(np.shape(train_1))
print(np.shape(train_2))
print(np.shape(real_class_num))
print(q1,q2)
print(tru_q1,tru_q2)


(559, 24)
(241, 24)
(100,)
0.69875 0.30125
0.705 0.295


In [17]:
est_mu1 = estimate_mean(train_1)
est_mu2 = estimate_mean(train_2)
est_sigma = estimate_sigma(train_1, train_2, p=p)
est_alpha = estimate_alpha(est_sigma, est_mu1, est_mu2)

z1 = calc_z(train_1, est_alpha)
z2 = calc_z(train_2, est_alpha)
cl_1, cl_2, pred_class_num = classify(test, z1, z2, q1, q2, est_alpha)

real_class_num = np.concatenate((np.ones(1+int(n * q1)), 2 * np.ones(int(n * q2))))

In [18]:
s_z = calc_s_z(est_alpha, est_sigma)
D = mahalanobis_distance(z1, z2, s_z)
D_H = unbiased_mahalanobis_distance(z1, z2, s_z, n1, n2, p=p)
print(D)
print(D_H)

K = np.log(q2 / q1)
prob_2_1 = stat.norm.cdf((K - 0.5 * D_H**2 ) / D_H)
prob_1_2 = stat.norm.cdf((-K - 0.5 * D_H**2 ) / D_H)
print(prob_2_1)
print(prob_1_2)

1.6367857536019372
2.4526180412547984
0.05828301183898339
0.18854617876434449


In [19]:

#            | predicted 1 | predicted 2 
# true val 1 |    [0,0]    |   [0,1]
# true val 2 |    [1,0]    |   [1,1]
table = print_error_table(pred_class_num, real_class_num)

[0.6300000000000004, 0.06999999999999999]
[0.15000000000000005, 0.15000000000000005]


In [20]:
N = np.shape(data)[0]
train = data[:int(N*0.8)]
test = data[int(N*0.8):]
train_1 = select_train_by_class(train, 1)
n1 = np.shape(train_1)[0]
train_2 = select_train_by_class(train, 2)
n2 = np.shape(train_2)[0]
test, tru_q1, tru_q2, n = process_test(test)


no_marks = np.concatenate((train_1,train_2,test), axis=0)
print(np.shape(no_marks))


pca = PCA(n_components=5)
data_pca = pca.fit(no_marks).transform(no_marks)
print(np.shape(data_pca))
train_1 = data_pca[:n1]
train_2 = data_pca[n1:n1+n2]
test = data_pca[n1+n2:]
p = np.shape(train_1)[1]
print(pca.explained_variance_ratio_)

est_mu1 = estimate_mean(train_1)
est_mu2 = estimate_mean(train_2)
est_sigma = estimate_sigma(train_1, train_2, p=p)
est_alpha = estimate_alpha(est_sigma, est_mu1, est_mu2)

z1 = calc_z(train_1, est_alpha)
z2 = calc_z(train_2, est_alpha)
cl_1, cl_2, pred_class_num = classify(test, z1, z2, q1, q2, est_alpha)

real_class_num = np.concatenate((np.ones(1+int(n * q1)), 2 * np.ones(int(n * q2))))
s_z = calc_s_z(est_alpha, est_sigma)
D = mahalanobis_distance(z1, z2, s_z)
D_H = unbiased_mahalanobis_distance(z1, z2, s_z, n1, n2, p=p)
print(D)
print(D_H)

K = np.log(q2 / q1)
prob_2_1 = stat.norm.cdf((K - 0.5 * D_H**2 ) / D_H)
prob_1_2 = stat.norm.cdf((-K - 0.5 * D_H**2 ) / D_H)
print(prob_2_1)
print(prob_1_2)
table = print_error_table(pred_class_num, real_class_num)

(1000, 24)
(1000, 5)
[0.79426932 0.12054817 0.07462957 0.00251035 0.00147001]
1.0543153930267173
1.0733900384210324
0.09333032535338298
0.597597118737295
[0.6250000000000004, 0.075]
[0.14000000000000004, 0.16000000000000006]


In [22]:
import plotly.express as px
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    data_pca,
    labels=labels,
    dimensions=range(4),
    color=data[:,-1]
)
fig.update_traces(diagonal_visible=False)
fig.show()